In [1]:
from perses.protocols.nonequilibrium_cycling import SimulationUnit, ResultUnit
from gufe import ChemicalSystem, SmallMoleculeComponent, ProteinComponent, SolventComponent


Level 5:azure.storage.blob._shared.avro.schema:Register new name for 'org.apache.avro.file.Header'
Level 5:azure.storage.blob._shared.avro.schema:Register new name for 'org.apache.avro.file.magic'
Level 5:azure.storage.blob._shared.avro.schema:Register new name for 'org.apache.avro.file.sync'


In [2]:
protein_comp = ProteinComponent.from_pdb_file("/home/ijpulidos/workdir/repos/perses/examples/protein-ligand/neq-cycling/Tyk2_protein.pdb")
ligand_a_comp = SmallMoleculeComponent.from_sdf_file('/home/ijpulidos/workdir/repos/perses/examples/protein-ligand/neq-cycling/ejm_31.sdf')
ligand_b_comp = SmallMoleculeComponent.from_sdf_file('/home/ijpulidos/workdir/repos/perses/examples/protein-ligand/neq-cycling/jmc_30.sdf')
state_a_full = {'protein': protein_comp, 'ligand': ligand_a_comp}
state_b_full = {'protein': protein_comp, 'ligand': ligand_b_comp}
state_a_ligand = {'ligand': ligand_a_comp}
state_b_ligand = {'ligand': ligand_b_comp}

system_a = ChemicalSystem(components=state_a_ligand)
system_b = ChemicalSystem(components=state_b_ligand)


In [3]:
# mapping class implementation

import gufe
from typing import Any, Optional
import numpy as np
from numpy.typing import NDArray
import json

class LigandAtomMapping(gufe.AtomMapping):
    """Simple container with the mapping between two Molecules

    """
    def __init__(
        self,
        componentA: SmallMoleculeComponent,
        componentB: SmallMoleculeComponent,
        componentA_to_componentB: dict[int, int],
        annotations: Optional[dict[str, Any]] = None,
    ):
        """
        Parameters
        ----------
        componentA, componentB : SmallMoleculeComponent
          the ligand molecules on either end of the mapping
        componentA_to_componentB : dict[int, int]
          correspondence of indices of atoms between the two ligands
        annotations : dict[str, Any]
          Mapping of annotation identifier to annotation data. Annotations may
          contain arbitrary JSON-serializable data. Annotation identifiers
          starting with ``ofe-`` may have special meaning in other parts of
          OpenFE. ``score`` is a reserved annotation identifier.
        """
        super().__init__(componentA, componentB)
        self._compA_to_compB = componentA_to_componentB

        if annotations is None:
            # TODO: this should be a frozen dict
            annotations = {}

        self._annotations = annotations

    @classmethod
    def _defaults(self):
        return {}

    @property
    def componentA_to_componentB(self) -> dict[int, int]:
        return dict(self._compA_to_compB)

    @property
    def componentB_to_componentA(self) -> dict[int, int]:
        return {v: k for k, v in self._compA_to_compB.items()}

    @property
    def componentA_unique(self):
        return (i for i in range(self.componentA.to_rdkit().GetNumAtoms())
                if i not in self._compA_to_compB)

    @property
    def componentB_unique(self):
        return (i for i in range(self.componentB.to_rdkit().GetNumAtoms())
                if i not in self._compA_to_compB.values())

    @property
    def annotations(self):
        # return a copy (including copy of nested)
        return json.loads(json.dumps(self._annotations))

    def _to_dict(self):
        """Serialize to dict"""
        return {
            # openff serialization doesn't go deep, so stringify at this level
            'componentA': json.dumps(self.componentA.to_dict(), sort_keys=True),
            'componentB': json.dumps(self.componentB.to_dict(), sort_keys=True),
            'componentA_to_componentB': self._compA_to_compB,
            'annotations': json.dumps(self.annotations),
        }

    @classmethod
    def _from_dict(cls, d: dict):
        """Deserialize from dict"""
        # the mapping dict gets mangled sometimes
        mapping = d['componentA_to_componentB']
        fixed = {int(k): int(v) for k, v in mapping.items()}

        return cls(
            componentA=SmallMoleculeComponent.from_dict(
                json.loads(d['componentA'])),
            componentB=SmallMoleculeComponent.from_dict(
                json.loads(d['componentB'])),
            componentA_to_componentB=fixed,
            annotations=json.loads(d['annotations'])
        )

    def __repr__(self):
        return (f"{self.__class__.__name__}(componentA={self.componentA!r}, "
                f"componentB={self.componentB!r}, "
                f"componentA_to_componentB={self._compA_to_compB!r}, "
                f"annotations={self.annotations!r})")

    def _ipython_display_(self, d2d=None):  # pragma: no-cover
        """
        Visualize atom mapping in a Jupyter Notebook.

        Parameters
        ---------
        d2d : :class:`rdkit.Chem.Draw.rdMolDraw2D.MolDraw2D`
            If desired specify an instance of a MolDraw2D object.
            Default ``None`` will use the MolDraw2DCairo backend.

        Returns
        -------
        Image: IPython.core.display.Image
            Image of the atom map
        """
        from IPython.display import Image, display

        return display(Image(draw_mapping(self._compA_to_compB,
                                          self.componentA.to_rdkit(),
                                          self.componentB.to_rdkit(), d2d)))

    def draw_to_file(self, fname: str, d2d=None):
        """
        Save atom map visualization to disk

        Parameters
        ---------
        d2d : :class:`rdkit.Chem.Draw.rdMolDraw2D.MolDraw2D`
            If desired specify an instance of a MolDraw2D object.
            Default ``None`` will write a .png file using the MolDraw2DCairo
            backend.

        fname : str
            Name of file to save atom map
        """
        data = draw_mapping(self._compA_to_compB, self.componentA.to_rdkit(),
                            self.componentB.to_rdkit(), d2d)
        if type(data) == bytes:
            mode = "wb"
        else:
            mode = "w"

        with open(fname, mode) as f:
            f.write(draw_mapping(self._compA_to_compB, self.componentA.to_rdkit(),
                                 self.componentB.to_rdkit(), d2d))

    def with_annotations(self, annotations: dict[str, Any]):
        """Create an new mapping based on this one with extra annotations.

        Parameters
        ----------
        annotations : dict[str, Any]
            Annotation update for this mapping. New annotation keys will be
            added to the annotations dict; existing keys will be replaced by
            the data provided here.
        """
        return self.__class__(
            componentA=self.componentA,
            componentB=self.componentB,
            componentA_to_componentB=self._compA_to_compB,
            annotations=dict(**self.annotations, **annotations)
        )

    def get_distances(self) -> NDArray[np.float64]:
        """Return the distances between pairs of atoms in the mapping"""
        dists = []
        molA = self.componentA.to_rdkit().GetConformer()
        molB = self.componentB.to_rdkit().GetConformer()
        for i, j in self._compA_to_compB.items():
            dA = molA.GetAtomPosition(i)
            dB = molB.GetAtomPosition(j)
            dists.append(dA.Distance(dB))

        return np.array(dists)

In [4]:
# Build gufe mapping object
from gufe.mapping.atom_mapping import AtomMapping
# Extracted from perses AtomMapper
mapping_dict = {0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 23: 20,
 26: 22,
 27: 23,
 28: 24,
 29: 25,
 30: 26,
 31: 27,
 32: 28,
 33: 29}
mapping = LigandAtomMapping(
    componentA=system_a.components['ligand'],
    componentB=system_b.components['ligand'],
    componentA_to_componentB=mapping_dict)

In [5]:
from openff.units import unit
# Build Settings gufe object
from gufe.settings.models import (
    Settings, 
    ThermoSettings, 
    ForcefieldSettings,
)
from perses.protocols.settings import NonEqCyclingSettings
settings = Settings(
    settings_version=0,
    forcefield_file="foobar.xml", 
    forcefield_settings=ForcefieldSettings(),
    thermo_settings=ThermoSettings(temperature=300*unit.kelvin),
    protocol_settings=NonEqCyclingSettings(),
)
# non_eq_settings = settings.NonEqCyclingSettings()

In [6]:
settings.json()

'{"settings_version": 0, "forcefield_file": "foobar.xml", "forcefield_settings": {"date": null, "author": null, "vdW": {"combining_rules": "Lorentz-Berthelot", "potential": "Lennard-Jones-12-6", "scale": {"scale12": 0.0, "scale13": 0.0, "scale14": 0.5, "scale15": 1.0}, "long_range_dispersion": "isotropic", "cutoff": 9.0, "switch_width": 1.0, "method": "cutoff"}, "electrostatics": {"periodic_potential": "Ewald3D-ConductingBoundary", "nonperiodic_potential": "Coulomb", "exception_potential": "Coulomb", "scale": {"scale12": 0.0, "scale13": 0.0, "scale14": 0.833333, "scale15": 1.0}, "cutoff": null, "switch_width": null, "solvent_dielectric": null}, "gbsa": {"gb_model": "OBC1", "solvent_dielectric": 78.5, "solute_dielectric": 1}}, "thermo_settings": {"temperature": "{\\"val\\": 300, \\"unit\\": \\"kelvin\\"}", "pressure": null, "ph": null, "redox_potential": null}, "protocol_settings": {"lambda_functions": {"lambda_sterics_core": "lambda", "lambda_electrostatics_core": "lambda", "lambda_ste

In [7]:
settings.dict()

{'settings_version': 0,
 'forcefield_file': 'foobar.xml',
 'forcefield_settings': {'date': None,
  'author': None,
  'vdW': {'combining_rules': 'Lorentz-Berthelot',
   'potential': 'Lennard-Jones-12-6',
   'scale': {'scale12': 0.0, 'scale13': 0.0, 'scale14': 0.5, 'scale15': 1.0},
   'long_range_dispersion': 'isotropic',
   'cutoff': 9.0,
   'switch_width': 1.0,
   'method': 'cutoff'},
  'electrostatics': {'periodic_potential': 'Ewald3D-ConductingBoundary',
   'nonperiodic_potential': 'Coulomb',
   'exception_potential': 'Coulomb',
   'scale': {'scale12': 0.0,
    'scale13': 0.0,
    'scale14': 0.833333,
    'scale15': 1.0},
   'cutoff': None,
   'switch_width': None,
   'solvent_dielectric': None},
  'gbsa': {'gb_model': 'OBC1',
   'solvent_dielectric': 78.5,
   'solute_dielectric': 1}},
 'thermo_settings': {'temperature': 300 <Unit('kelvin')>,
  'pressure': None,
  'ph': None,
  'redox_potential': None},
 'protocol_settings': {'lambda_functions': {'lambda_sterics_core': 'lambda',
   '

In [6]:
simulation = SimulationUnit(state_a=system_a, state_b=system_b, mapping=mapping, settings=non_eq_settings)
result = simulation.execute(shared='/tmp/', **simulation.inputs)


2022-11-29 09:21:01,032:(0.00s):utils.openeye:molecule ejm_31                      does not have unique atom names. Generating now...
2022-11-29 09:21:01,032:(0.00s):utils.openeye:molecule jmc_30                      does not have unique atom names. Generating now...
2022-11-29 09:21:01,033:(0.00s):relative_setup:			_hmass: 3 Da.

2022-11-29 09:21:01,033:(0.00s):relative_setup:No receptor specified. Assuming non-complex phase.
2022-11-29 09:21:01,034:(0.00s):relative_setup:	setting both molecule oemol titles to 'MOL'.
2022-11-29 09:21:01,035:(0.00s):relative_setup:Created mdtraj topologies for both ligands.
2022-11-29 09:21:01,036:(0.00s):relative_setup:Setting non bonded method to PME
2022-11-29 09:21:01,037:(0.00s):relative_setup:set MonteCarloBarostat because pressure was specified as 1.0 atm atmospheres
2022-11-29 09:21:01,044:(0.01s):relative_setup:PME tolerance: 0.00025


Receptor not found in chemical system. Assuming non-complex phase for system.
Receptor not found in chemical system. Assuming non-complex phase for system.


2022-11-29 09:21:01,307:(0.26s):relative_setup:successfully created SystemGenerator to create ligand systems
2022-11-29 09:21:01,308:(0.00s):relative_setup:executing SmallMoleculeSetProposalEngine...
2022-11-29 09:21:01,309:(0.00s):relative_setup:instantiating FFAllAngleGeometryEngine...
2022-11-29 09:21:01,309:(0.00s):relative_setup:Detected solvent...
2022-11-29 09:21:01,309:(0.00s):relative_setup:assgning noCutoff to nonbonded_method
2022-11-29 09:21:01,310:(0.00s):relative_setup:Removing barostat for vacuum phase
2022-11-29 09:21:01,310:(0.00s):relative_setup:Setting nonbondedMethod to NoCutoff for vacuum phase
2022-11-29 09:21:01,310:(0.00s):relative_setup:calling SystemGenerator to create ligand systems.
2022-11-29 09:21:01,310:(0.00s):relative_setup:No complex or solvent leg, so performing topology proposal for vacuum leg
2022-11-29 09:21:01,311:(0.00s):relative_setup:	Skipping solvation of vacuum perturbation
2022-11-29 09:21:01,311:(0.00s):relative_setup:	parameterizing...
202

2022-11-29 09:21:13,868:(0.00s):geometry:neglected angle terms include []
2022-11-29 09:21:13,869:(0.00s):geometry:log probability choice of torsions and atom order: -11.597854675755533
2022-11-29 09:21:13,869:(0.00s):geometry:creating platform, integrators, and contexts; setting growth parameter
2022-11-29 09:21:13,876:(0.01s):geometry:setting atoms_with_positions context new positions
2022-11-29 09:21:13,880:(0.00s):geometry:There are 9 new atoms
2022-11-29 09:21:13,883:(0.00s):geometry:	reduced angle potential = 0.1269853496996263.
2022-11-29 09:21:13,911:(0.03s):geometry:	reduced angle potential = 0.1386446585974545.
2022-11-29 09:21:13,936:(0.03s):geometry:	reduced angle potential = 1.406463107003945.
2022-11-29 09:21:13,959:(0.02s):geometry:	reduced angle potential = 0.05535592133136959.
2022-11-29 09:21:13,982:(0.02s):geometry:	reduced angle potential = 1.1181430510065242.
2022-11-29 09:21:14,007:(0.02s):geometry:	reduced angle potential = 0.19310528381898276.
2022-11-29 09:21:1

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


2022-11-29 09:21:14,079:(0.03s):geometry:	reduced angle potential = 0.008298441175535413.
2022-11-29 09:21:14,106:(0.03s):geometry:	beginning construction of no_nonbonded final system...
2022-11-29 09:21:14,107:(0.00s):geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
2022-11-29 09:21:14,109:(0.00s):geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2022-11-29 09:21:14,110:(0.00s):geometry:	there are 27 bond forces in the no-nonbonded final system
2022-11-29 09:21:14,110:(0.00s):geometry:	there are 65 angle forces in the no-nonbonded final system
2022-11-29 09:21:14,110:(0.00s):geometry:	there are 153 torsion forces in the no-nonbonded final system
2022-11-29 09:21:14,110:(0.00s):geometry:forward final system defined with 0 neglected angles.
2022-11-29 09:21:14,127:(0.02s):geometry:total reduced potential b

2022-11-29 09:21:14,295:(0.00s):relative:Adding angle force terms...
2022-11-29 09:21:14,295:(0.00s):relative:Adding torsion force terms...
2022-11-29 09:21:14,295:(0.00s):relative:Adding nonbonded force terms...
2022-11-29 09:21:14,295:(0.00s):relative:	_add_nonbonded_force_terms: <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7f7accffd7a0> > added to hybrid system
2022-11-29 09:21:14,295:(0.00s):relative:	_add_nonbonded_force_terms: nonbonded_method is NoCutoff
2022-11-29 09:21:14,296:(0.00s):relative:	_add_nonbonded_force_terms: 0 added to standard nonbonded force
2022-11-29 09:21:14,304:(0.01s):relative:	_add_nonbonded_force_terms: 0 added to sterics_custom_nonbonded force
2022-11-29 09:21:14,305:(0.00s):relative:	_add_nonbonded_force_terms: <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x7f7accffd9b0> > added to hybrid system
2022-11-29 09:21:14,305:(0.00s):relative:Handling harm

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
added energy components: {'CustomBondForce': 163.62693393096956, 'CustomAngleForce': 191.09414510658723, 'CustomTorsionForce': 21.89977880333886}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
added energy components: {'CustomBondForce': -58.13460765349573, 'CustomAngleForce': 17.486668370697988, 'CustomTorsionForce': 1.05298

2022-11-29 09:21:14,363:(0.04s):relative:Handling unique_new/old interaction exceptions...
2022-11-29 09:21:14,363:(0.00s):relative:There are old or new system exceptions...proceeding.
2022-11-29 09:21:14,365:(0.00s):relative:Imposing virtual bonds to ensure system is imaged together.
2022-11-29 09:21:14,380:(0.02s):relative:		_impose_virtual_bonds: No restraint added because one set is empty (core_atoms=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20], protein_atoms=[])


In [7]:
result.ok()

True

In [8]:
print(result.traceback)

AttributeError: 'ProtocolUnitResult' object has no attribute 'traceback'

In [9]:
# GatherUnit -- Results
neq_results = ResultUnit(phase="vacuum")
# result = neq_results.execute(shared='/tmp/', **neq_results.inputs)
result = neq_results.execute(shared='/tmp/', **neq_results.inputs)

In [10]:
result.outputs

{}

In [6]:
# How to run the protocol
from perses.protocols.nonequilibrium_cycling import NonEquilibriumCycling
from gufe.protocols.protocoldag import execute
neq_cycling = NonEquilibriumCycling(settings)
dag_result = execute(neq_cycling.create(stateA=system_a, 
                                        stateB=system_b, 
                                        mapping=mapping,
                                       )
                    )

2022-11-29 18:53:09,932:(0.00s):utils.openeye:molecule ejm_31                      does not have unique atom names. Generating now...
2022-11-29 18:53:09,933:(0.00s):utils.openeye:molecule jmc_30                      does not have unique atom names. Generating now...
2022-11-29 18:53:09,933:(0.00s):relative_setup:			_hmass: 3 Da.

2022-11-29 18:53:09,934:(0.00s):relative_setup:No receptor specified. Assuming non-complex phase.
2022-11-29 18:53:09,935:(0.00s):relative_setup:	setting both molecule oemol titles to 'MOL'.
2022-11-29 18:53:09,936:(0.00s):relative_setup:Created mdtraj topologies for both ligands.
2022-11-29 18:53:09,937:(0.00s):relative_setup:Setting non bonded method to PME
2022-11-29 18:53:09,937:(0.00s):relative_setup:set MonteCarloBarostat because pressure was specified as 1.0 atm atmospheres
2022-11-29 18:53:09,945:(0.01s):relative_setup:PME tolerance: 0.00025


Receptor not found in chemical system. Assuming non-complex phase for system.
Receptor not found in chemical system. Assuming non-complex phase for system.


2022-11-29 18:53:10,313:(0.37s):relative_setup:successfully created SystemGenerator to create ligand systems
2022-11-29 18:53:10,313:(0.00s):relative_setup:executing SmallMoleculeSetProposalEngine...
2022-11-29 18:53:10,314:(0.00s):relative_setup:instantiating FFAllAngleGeometryEngine...
2022-11-29 18:53:10,314:(0.00s):relative_setup:Detected solvent...
2022-11-29 18:53:10,315:(0.00s):relative_setup:assgning noCutoff to nonbonded_method
2022-11-29 18:53:10,315:(0.00s):relative_setup:Removing barostat for vacuum phase
2022-11-29 18:53:10,315:(0.00s):relative_setup:Setting nonbondedMethod to NoCutoff for vacuum phase
2022-11-29 18:53:10,316:(0.00s):relative_setup:calling SystemGenerator to create ligand systems.
2022-11-29 18:53:10,316:(0.00s):relative_setup:No complex or solvent leg, so performing topology proposal for vacuum leg
2022-11-29 18:53:10,316:(0.00s):relative_setup:	Skipping solvation of vacuum perturbation
2022-11-29 18:53:10,317:(0.00s):relative_setup:	parameterizing...
202

2022-11-29 18:53:26,956:(0.00s):geometry:neglected angle terms include []
2022-11-29 18:53:26,957:(0.00s):geometry:log probability choice of torsions and atom order: -11.666847547242485
2022-11-29 18:53:26,958:(0.00s):geometry:creating platform, integrators, and contexts; setting growth parameter
2022-11-29 18:53:26,967:(0.01s):geometry:setting atoms_with_positions context new positions
2022-11-29 18:53:26,976:(0.01s):geometry:There are 9 new atoms
2022-11-29 18:53:26,981:(0.01s):geometry:	reduced angle potential = 0.036068134696983535.
2022-11-29 18:53:27,031:(0.05s):geometry:	reduced angle potential = 0.0006261299879561936.
2022-11-29 18:53:27,066:(0.03s):geometry:	reduced angle potential = 0.24946841997007105.
2022-11-29 18:53:27,114:(0.05s):geometry:	reduced angle potential = 0.26055999449016976.
2022-11-29 18:53:27,151:(0.04s):geometry:	reduced angle potential = 0.04871812117030455.


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


2022-11-29 18:53:27,191:(0.04s):geometry:	reduced angle potential = 0.12685305493503785.
2022-11-29 18:53:27,229:(0.04s):geometry:	reduced angle potential = 0.004120171965160933.
2022-11-29 18:53:27,264:(0.03s):geometry:	reduced angle potential = 0.03137987875693876.
2022-11-29 18:53:27,305:(0.04s):geometry:	reduced angle potential = 0.002782386941724597.
2022-11-29 18:53:27,338:(0.03s):geometry:	beginning construction of no_nonbonded final system...
2022-11-29 18:53:27,339:(0.00s):geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
2022-11-29 18:53:27,341:(0.00s):geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2022-11-29 18:53:27,342:(0.00s):geometry:	there are 27 bond forces in the no-nonbonded final system
2022-11-29 18:53:27,342:(0.00s):geometry:	there are 65 angle forces in the no-nonbonded final syst

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
added energy components: {'CustomBondForce': 666.6390637526381, 'CustomAngleForce': 222.4704502956763, 'CustomTorsionForce': 40.704549155886426}
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


2022-11-29 18:53:27,563:(0.03s):geometry:	beginning construction of no_nonbonded final system...
2022-11-29 18:53:27,564:(0.00s):geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
2022-11-29 18:53:27,567:(0.00s):geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
2022-11-29 18:53:27,568:(0.00s):geometry:	there are 22 bond forces in the no-nonbonded final system
2022-11-29 18:53:27,568:(0.00s):geometry:	there are 52 angle forces in the no-nonbonded final system
2022-11-29 18:53:27,568:(0.00s):geometry:	there are 129 torsion forces in the no-nonbonded final system
2022-11-29 18:53:27,568:(0.00s):geometry:reverse final system defined with 0 neglected angles.
2022-11-29 18:53:27,578:(0.01s):geometry:total reduced potential before atom placement: 82.01720749728068
2022-11-29 18:53:27,594:(0.02s):geometry:total red

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
added energy components: {'CustomBondForce': -58.119818301886426, 'CustomAngleForce': 12.010081935566392, 'CustomTorsionForce': 3.4306500290168342}


In [7]:
dag_result.ok()

True

In [8]:
dag_result.protocol_unit_failures

[]

In [9]:
failure = dag_result.protocol_unit_failures[0]

IndexError: list index out of range

In [10]:
print(failure.traceback)

NameError: name 'failure' is not defined

In [11]:
a = dag_result.protocol_unit_results[0]

In [14]:
dag_result.protocol_unit_results[1].outputs

{'DDG': -33.62820884362574,
 'dDDG': 3.725290298461914e-09,
 'paths': {'forward_work': '/tmp/tmp54sbz1z6/forward_vacuum.npy',
  'reverse_work': '/tmp/tmp54sbz1z6/reverse_vacuum.npy'}}

In [18]:
result_graph = dag_result.result_graph

In [19]:
result_graph.nodes

NodeView((ProtocolUnitResult(None), ProtocolUnitResult(result)))

In [20]:
dag_result.graph.nodes

NodeView((SimulationUnit(None), ResultUnit(result)))